定制的优先队列`ChangeablePriorityQueue`，不仅要实现优先队列特性，还要支持降低元素的权重。

实现：
* 通过堆heap维护元素key的优先级
* 通过哈希表维护元素key的索引


接口：
* `get() -> Tuple[weight, key]` 元素出队
* `put(item: Tuple[weight, key])` 元素入队
* `decrease_key(key, weight)` 降低元素的权重

In [1]:
# encoding: utf8

from typing import Tuple, Any


class Empty(Exception):
    pass


class Full(Exception):
    pass


def _parent(pos: int):
    return (pos - 1) // 2 if pos > 0 else 0


def _left(pos: int):
    return 2 * pos + 1


def _right(pos: int):
    return 2 * pos + 2


class ChangeablePriorityQueue:

    def __init__(self, max_size: int = 0) -> None:
        self.queue = []
        self.map = {}
        self.max_size = max_size

    def heapify_up(self, i: int) -> None:
        p = _parent(i)    # type:int
        if p == i:
            return

        q = self.queue
        if q[p] > q[i]:
            q[p], q[i] = q[i], q[p]

            # 更新映射索引
            k1, k2 = q[p][1], q[i][1]
            self.map[k1], self.map[k2] = self.map[k2], self.map[k1]

            self.heapify_up(p)

    def heapify_down(self, i: int) -> None:
        if i >= len(self.queue):
            return

        q = self.queue
        least = i

        l = _left(i)
        if l < len(q) and q[l] < q[least]:
            least = l

        r = _right(i)
        if r < len(q) and q[r] < q[least]:
            least = r

        if least != i:
            q[least], q[i] = q[i], q[least]

            k1, k2 = q[least][1], q[i][1]
            self.map[k1], self.map[k2] = self.map[k2], self.map[k1]

            self.heapify_down(least)

    def get(self) -> Tuple[float, Any]:
        if not self.queue:
            raise Empty()

        item = self.queue[0]

        self.queue[0], self.queue[-1] = self.queue[-1], self.queue[0]
        self.map[self.queue[0][1]] = 0
        self.map[self.queue[-1][1]] = len(self.queue) - 1

        self.map.pop(item[1])
        self.queue.pop()

        self.heapify_down(0)

        return item

    def put(self, item: Tuple[float, Any]):
        if len(self.queue) == self.max_size:
            raise Full()

        weight, key = item
        self.queue.append(item)
        self.map[key] = len(self.queue) - 1

        self.heapify_up(len(self.queue)-1)

    def decrease_key(self, key: Any, weight: float):
        assert key in self.map

        self.queue[self.map[key]] = (weight, key)

        self.heapify_up(self.map[key])
